This notebook tries out the state-of-the-art word embeding model [ELMo](https://allennlp.org/elmo).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import pandas as pd

from config import validate_data_path, train_data_path, testa_data_path, testb_data_path
from fgclassifier import read_csv

def content_to_corpus(data_path, txt_path, sample_n=None, print_sample=10):
    """Convert Review content to text corpus for word embedding training"""
    df = read_csv(data_path, seg_words=True, sample_n=None)
    if sample_n:
        df = df.sample(sample_n, random_state=1)
    sentences = [
        y.strip() for x in df['content']
        for y in x.strip('"').split('。') if y.strip()
    ]
    if print_sample:
        print('\n'.join(sentences[:print_sample]))
    all_content = '\n'.join(sentences)
    with open(txt_path, 'w') as f:
        f.write(all_content + '\n')

In [3]:
content_to_corpus(train_data_path, 'data/text_train.txt')

2018-11-15 02:38:27,079 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..


吼吼 吼 ， 萌死 人 的 棒棒糖 ， 中 了 大众 点评 的 霸王餐 ， 太 可爱 了
一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ， 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会
看 介绍 棒棒糖 是 用 德国 糖 做 的 ， 不会 很甜 ， 中间 的 照片 是 糯米 的 ， 能 食用 ， 真是太 高端 大气 上档次 了 ， 还 可以 买 蝴蝶结 扎口 ， 送人 可以 买 礼盒
我 是 先 打 的 卖家 电话 ， 加 了 微信 ， 给 卖家 传 的 照片
等 了 几天 ， 卖家 就 告诉 我 可以 取货 了 ， 去 大官 屯 那取 的
虽然 连 卖家 的 面 都 没 见到 ， 但是 还是 谢谢 卖家 送 我 这么 可爱 的 东西 ， 太 喜欢 了 ， 这 哪 舍得吃 啊
第三次 参加 大众 点评 网 霸王餐 的 活动
这家 店 给 人 整体 感觉 一般
首先 环境 只能 算 中等 ， 其次 霸王餐 提供 的 菜品 也 不是 很多 ， 当然 商家 为了 避免 参加 霸王餐 吃不饱 的 现象 ， 给 每桌 都 提供 了 至少 六份 主食 ， 我们 那桌 都 提供 了 两份 年糕 ， 第一次 吃火锅 会 在 桌上 有 这么 多 的 主食 了
整体 来说 这家 火锅店 没有 什么 特别 有 特色 的 ， 不过 每份 菜品 分量 还是 比较 足 的 ， 这点 要 肯定 ！ 至于 价格 ， 因为 没有 看 菜单 不 了解 ， 不过 我 看 大众 有 这家 店 的 团购 代金券 ， 相当于 7 折 ， 应该 价位 不会 很 高 的 ！ 最后 还是 要 感谢 商家 提供 霸王餐 ， 祝 生意兴隆 ， 财源 广进


In [8]:
content_to_corpus(train_data_path, 'data/text_train_10k.txt',
                  sample_n=10000, print_sample=False)

2018-11-15 02:39:32,887 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..


In [9]:
content_to_corpus(validate_data_path, 'data/text_valid.txt')

2018-11-15 02:39:37,706 [INFO] Read cache data/validate/sentiment_analysis_validationset.csv.segged_sample_None.tsv..


哎 ， 想当年 来 佘山 的 时候 ， 啥 都 没有 ， 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了
菜品 多 ， 有点 太 多 ， 感觉 啥 都 有 ， 杂都 不足以 形容
随便 点些 ， 居然 口味 什么 的 都 好 还 可以 ， 价钱 自然 是 便宜 当 震惊
元宝 虾 和 椒盐 九肚鱼 都 不错 吃
不过 近来 几次 么 ， 味道 明显 没 以前 好 了
冷餐 里面 一个 凉拌 海带丝 还 可以 ， 酸酸甜甜 的
镇上 也 有 了 些 别的 大点 的 饭店 ， 所以 不是 每次 必来 了
对 了 ， 这家 的 生意 一如既往 的 超级 好 ， 不 定位 基本 吃 不到
不过 佘山 这边 的 人 吃晚饭 很早 的 ， 所以 稍微 晚点 去 就 很 空 了
趁着 国庆节 ， 一家人 在 白天 在 山里 玩耍 之后 ， 晚上 决定 吃 李记 搅团


In [10]:
content_to_corpus(testa_data_path, 'data/text_testa.txt')

2018-11-15 02:39:42,945 [INFO] Read cache data/test-a/sentiment_analysis_testa.csv.segged_sample_None.tsv..


我 想 说 他们 家 的 优惠活动 好 持久 啊 ， 我 预售 的 时候 买 的 券 ， 前两天 心血来潮 去 吃 的 活动 还 在 继续
首先 说 下 服务 ， 因为 和 男票 开车 去 的 ， 有点 不 认路 ， 老板 很 耐心 的 在 电话 里 帮 我们 指路 ， 到 了 门店 之后 也 帮 我们 推荐 了 他们 家 做 的 比较 地道 的 伤心 凉粉 ， 说 是 厨师 是 四川 那边 来 的
环境 呢 比较简单 干净 ， 去 的 时候 下午 一点多 了 ， 还有 四五桌 人 在 用餐
口味 对于 我 而言 点 了 麻辣 的 口感 正 正好 ， 男票 比较 能 吃 辣 ， 相对而言 觉得 他们 家 的 麻辣 口感 麻有 了 ， 辣 还 欠缺 一点 ， 老板娘 说 考虑 到 客人 口味 不同 所以 没敢 放太多 辣椒 ， 能 吃 辣 的 朋友 可以 考虑 下单 之前 和 老板 先 说好
鱼 呢 我们 选 的 是 黑鱼 ， 2.9 斤 的 鱼 加上 一盆 我 以为 没有 什么 东西 实际上 东西 很多 的 锅底 ， 我们 吃 的 饱饱 的 ， 最后 以为 吃 的 差不多 了 ， 打包 一看 简直 像 没动 过 一样 ， 分量 还是 满足 的 ， 鱼 比较 新鲜
伤心 凉粉 很辣 ， 不过 口味 也 蛮 好吃 的
总的来说 ， 性价比 还是 可以 的 ， 两个 人 吃 了 大概 160 左右 ， 用 了 团购 券 的话 一百块 不到 ， 会 考虑 下次 再 来
终于 开 到 心心念念 的 LAB BBLANKK loft
第一次 来 就 随便 点 也 一些 ～ 【 香辣虾 意 面 】 蛮辣 的 ， 但 其实 一般般
【 玛格丽特 】 进口 的 感觉 蛮 好 的 就是 喝完 后 就 点 呛 ～ 但是 朋友 不是 很 喜欢 【 一柱擎天 】 看 点评 很多 人 说 喜欢 就 点 了 ， 水蜜桃 味 ， 还 不错 挺 好喝 的 ～ 赞 【 海鲜 饭 】 想 吃饭 但 这店 的 饭类 只有 两种 ， 就 点 了 这个


In [11]:
content_to_corpus(testb_data_path, 'data/text_testb.txt')

2018-11-15 02:39:47,189 [INFO] Reading data/test-b/sentiment_analysis_testb.csv..
2018-11-15 02:39:50,189 [INFO] Segmenting data/test-b/sentiment_analysis_testb.csv..
100%|██████████| 200000/200000 [07:57<00:00, 419.20it/s]
2018-11-15 02:47:51,128 [INFO] Saved cached data/test-b/sentiment_analysis_testb.csv.segged_sample_None.tsv.


可以 说 工作日 中午 的 这个 套餐 着实 是 特别 的 实惠 啊 ， 店家 给 的 量 也 是 大大的 足 ！ 首先 如图 ， 每个 菜品 摆盘 都 很 精美 ， 让 人 一 看 就 充满 食欲
土豆泥 口味 很棒 ， 甜 酱油 、 鱼籽 和 沙拉酱 的 搭配 非常 完美
三文鱼 很 新鲜 ， 切得 得 好 厚 啊 ， 吃 起来 非常 满足 ！ 猪排 饭上 的 实在 有点 慢 了 ， 至少 等 了 20 分钟 ， 但量 超级 大 ， 上面 是 又 大 又 厚 的 炸 猪排 ， 不 知道 是不是 做好 了 没有 及时 给 上 ， 猪排 已经 被 酱汁 泡得变 湿软 了 ， 没有 了 酥脆 感
这个 套餐 店家 真的 是 满满的 诚意 啊 ， 估计 一般 人 真的 吃不完
位置 很 不错 ， 老板 眼光 很 好 👍
位于 地铁站 附近 ， 周围 有 办公楼 、 商圈 、 住宅 、 别墅 虽然 现在 还 有些 荒凉 ， 但 我 想 很快 会 发展 起来 的 💪
餐厅 内部 的 环境 还 算 干净 温馨
服务员 也 很 热情 亲切
姜油大 芥菜 ： 首先 看到 觉得 略微 粗旷 了 些 ， 但 吃 起来 味道 不错 ， 姜丝 没有 很 浓重 的 味道
香煎海 鲈鱼 ： 特别 酥脆 鲜香 好吃 😋


In [12]:
!cat data/text_train.txt data/text_valid.txt data/text_testa.txt data/text_testb.txt > data/text_all.txt
!wc data/text_all.txt

 2896571 76231079 412681013 data/text_all.txt


All dataset combined has 76 million words, which is too many. According to [ELMoForManyLangs](https://github.com/HIT-SCIR/ELMoForManyLangs), it would take 3 days to train 20m words on an NVIDIA P100 GPU.

We must sample the data.

In [16]:
!wc data/text_train_10k.txt

   82727 2255688 12200210 data/text_train_10k.txt


A random sample of 10,000 reviews gives about 83k sentences, and 2.2m words.
It should take probably 8 hours to train the embedings from the 10,000 reviews.

In [16]:
!tail data/text_train_10k.txt

店里 冰淇淋 有 12 个 口味 的 ~
除了 冰淇淋 ~ 甜品 饮料 也 有 ~ 看 电影 前 可以 来 这里 等 ~ 也 可以 买 进去 吃 哈 ~
# 枫树 胡桃 # BBLANKK
店员 妹子 挖 完 冰淇淋 还 帮 我 称 了 一下 ~ 说 他们 家 的 单球 是 60g ~
味道 很 好 啊 ~ 奶味算重 的 了 ~ 也 不会 很甜 ~ 里面 有 核桃仁 ~ 核桃仁 也 好吃 ~
一个 球挺 多 的 ~ 吃 着 绝对 够 了 ~
不过 原价 有点 小贵 ~ M 团有 团购 能 便宜 一点 ~
总体 很 满意 的 ~ 以后 想 吃 冰淇淋 可以 来 这家 ~ 再 吃 吃 别的 味 ~
极食 urban BBLANKK harvest 是 一个 走 自然 环保 高端 线路 的 餐厅 ， 位于 杭州 大厦 D座 5 楼 ～ 光看 地标 也 是 够 逼格 了 一进 门口 装饰 有 微型 田园 BBLANKK 看上去 食材 都 新鲜 的 不要 不要 的 ～ 给 创意 一个 赞 BBLANKK 只要 菜单 上 有 “ 即 摘 ” 字眼 的 ， 都 由 服务员 现场 采摘 取用 ， 菜色 摆盘 精致 而且 色彩 丰富 ， 让 人 食欲 大开 ， 口味 都 还 不错 ～ 不过 本人 本 就 不是 一个 太 挑食 的 人 啦 饭后 上 了 甜品 ， 好吃 到 飞 起来 ， 吃 得 愉快 心情 就 好好 ， 尤其 是 这样 一个 周末
因为 价格 偏贵 的 缘故 ， 如此 黄金地段 、 高峰 时间 用餐 人 倒 不 多 ， 这 也 保证 了 用餐 环境 的 舒适度 ， 还有 一个 好消息 就是 ： 开业 初期 有 七折 优惠 哦 ！ 大家 赶紧 去 拔草 吧




Let's try a very small subset just to make sure the software works.

Use 1000 sentences as training and 100 sentences as validation.

In [22]:
# !shuf -n 10000 data/all_text.txt > data/little_text.txt
# Use `gshuf` on Mac
!gshuf -n 1000 data/text_train.txt > data/text_train_1ks.txt
!gshuf -n 100 data/text_train.txt > data/text_train_100s.txt
!wc data/text_train_1ks.txt
!wc data/text_train_100s.txt

    1000   27705  150876 data/text_train_1ks.txt
     100    2729   14924 data/text_train_100s.txt


In [21]:
!python -m elmoformanylangs.biLM train \
    --train_path data/text_train_100s.txt \
    --model data/elmo-zhs-1k \
    --valid_size 10 \
    --config_path `pwd`/misc/elmo/cnn_50_100_512_4096_sample.json \
    --seed 1 \
    --gpu 0 \
    --optimizer adam \
    --lr 0.001 \
    --lr_decay 0.8 \
    --batch_size 32 \
    --clip_grad 5 \
    --max_epoch 10 \
    --max_sent_len 20 \
    --max_vocab_size 150000 \
    --eval_steps 10000 \
    --min_count 5

Namespace(batch_size=32, clip_grad=5.0, config_path='/Users/jesse/workspace/ML/fine-grain-sentiment-analysis/misc/elmo/cnn_50_100_512_4096_sample.json', eval_steps=10000, gpu=0, lr=0.001, lr_decay=0.8, max_epoch=10, max_sent_len=20, max_vocab_size=150000, min_count=5, model='data/elmo-zhs-1k', optimizer='adam', save_classify_layer=False, seed=1, test_path=None, train_path='data/text_train_100s.txt', valid_path=None, valid_size=10, word_embedding=None)
{'encoder': {'name': 'elmo', 'projection_dim': 512, 'cell_clip': 3, 'proj_clip': 3, 'dim': 4096, 'n_layers': 2}, 'token_embedder': {'name': 'cnn', 'activation': 'relu', 'filters': [[1, 32], [2, 32], [3, 64], [4, 128], [5, 256], [6, 512], [7, 1024]], 'n_highway': 2, 'word_dim': 100, 'char_dim': 50, 'max_characters_per_token': 50}, 'classifier': {'name': 'sampled_softmax', 'n_samples': 8192}, 'dropout': 0.1}
2018-11-15 03:06:43,847 INFO: training instance: 149, training tokens: 2836.
2018-11-15 03:06:43,847 INFO: training instance: 139, tra

In [24]:
# Train
# add `pwd` to `config_path` because trained model depends on that file
# absolute path makes it easier to find
!python -m elmoformanylangs.biLM train \
    --train_path data/text_train_1ks.txt \
    --valid_path data/text_train_100s.txt \
    --model data/elmo-zhs-1k \
    --config_path `pwd`/misc/elmo/cnn_50_100_512_4096_sample.json \
    --seed 1 \
    --gpu 0 \
    --optimizer adam \
    --lr 0.001 \
    --lr_decay 0.8 \
    --batch_size 32 \
    --clip_grad 5 \
    --max_epoch 10 \
    --max_sent_len 20 \
    --max_vocab_size 150000 \
    --eval_steps 10000 \
    --min_count 5

Namespace(batch_size=32, clip_grad=5.0, config_path='/Users/jesse/workspace/ML/fine-grain-sentiment-analysis/misc/elmo/cnn_50_100_512_4096_sample.json', eval_steps=10000, gpu=0, lr=0.001, lr_decay=0.8, max_epoch=10, max_sent_len=20, max_vocab_size=150000, min_count=5, model='data/elmo-zhs-1k', optimizer='adam', save_classify_layer=False, seed=1, test_path=None, train_path='data/text_train_1k.txt', valid_path='data/text_train_100.txt', valid_size=0, word_embedding=None)
{'encoder': {'name': 'elmo', 'projection_dim': 512, 'cell_clip': 3, 'proj_clip': 3, 'dim': 4096, 'n_layers': 2}, 'token_embedder': {'name': 'cnn', 'activation': 'relu', 'filters': [[1, 32], [2, 32], [3, 64], [4, 128], [5, 256], [6, 512], [7, 1024]], 'n_highway': 2, 'word_dim': 100, 'char_dim': 50, 'max_characters_per_token': 50}, 'classifier': {'name': 'sampled_softmax', 'n_samples': 8192}, 'dropout': 0.1}
2018-11-14 22:36:57,640 INFO: training instance: 1505, training tokens: 28596.
2018-11-14 22:36:57,645 INFO: valid i

In [25]:
df_train = read_csv(train_data_path, seg_words=True, sample_n=None)

2018-11-14 23:19:31,908 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..


In [28]:
from elmoformanylangs import Embedder

elmoemb = Embedder('data/elmo-zhs-fsauor')
# e = Embedder('data/elmo-zhs-1k')

sents = df_train['content'][0:2]
elmoemb.sents2elmo(sents)

2018-11-14 23:21:26,090 [INFO] char embedding size: 8582
2018-11-14 23:21:26,861 [INFO] word embedding size: 69827
2018-11-14 23:21:34,862 [INFO] Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(69827, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(8582, 50, padding_idx=8579)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_f

[array([[ 0.16197295,  0.05117923, -0.13360588, ...,  0.24924016,
         -0.27383432, -0.05000202],
        [ 0.06199374,  0.07696173, -0.23577517, ...,  0.26290917,
         -0.27809614, -0.06464732],
        [ 0.0008731 , -0.04712557, -0.08987609, ..., -0.00968068,
         -0.13784361, -0.03070146],
        ...,
        [-0.27461815,  0.03719755, -0.13428287, ..., -0.06027537,
         -0.07961495, -0.14139868],
        [-0.06108154,  0.13908525, -0.2571093 , ...,  0.01645198,
         -0.02265188, -0.02150639],
        [-0.19907278,  0.02419554, -0.19596738, ...,  0.07171986,
         -0.01507132,  0.02613006]], dtype=float32),
 array([[-0.17354196, -0.36698878, -0.04885902, ..., -0.02680009,
          0.13608061,  0.12091058],
        [-0.29550084, -0.12204532, -0.28084293, ..., -0.29321548,
          0.13516046, -0.11846065],
        [-0.05146622, -0.19961458,  0.01509347, ...,  0.05483557,
         -0.06374971, -0.0203303 ],
        ...,
        [-0.1181826 ,  0.02767085, -0.3

OK, now let's train the full model.

In [23]:
!cat data/text_train.txt data/text_testa.txt data/text_testb.txt > data/text_train_testab.txt
!wc data/text_train_testab.txt
!wc data/text_valid.txt

 2773187 72830536 394294246 data/text_train_testab.txt
  123384 3400543 18386767 data/text_valid.txt


In [ ]:
# To train the full model
!time python -m elmoformanylangs.biLM train \
    --train_path data/text_train_testab.txt \
    --valid_path data/text_valid.txt \
    --model data/elmo-zhs \
    --config_path `pwd`/misc/elmo/cnn_50_100_512_4096_sample.json \
    --seed 1 \
    --gpu 0 \
    --optimizer adam \
    --lr 0.001 \
    --lr_decay 0.8 \
    --batch_size 32 \
    --clip_grad 5 \
    --max_epoch 10 \
    --max_sent_len 20 \
    --max_vocab_size 150000 \
    --eval_steps 10000 \
    --min_count 5